# Homework 7

## APMTH 207: Stochastic Methods for Data Analysis, Inference and Optimization

##### Data: kcancer.csv

**Due Date: ** Thursday, March 23th, 2017 at 11:59pm

**Instructions:**

- Upload your final answers as well as your iPython notebook containing all work to Canvas.

- Structure your notebook and your work to maximize readability.

## Problem 1: Kidney Cancer Data Revisited
In this problem we will work with the US Kidney Cancer Data set (by county). This is a dataset of kidney cancer frequencies across the US over 5 years on a per county basis. 

In lecture, we've discussed the potential (and significant) draw backs of drawing conclusions based on raw cancer rates. Addressing these draw backs, we proposed using a Bayesian model to model the data. In the following, you will implement two Bayesian models and examine the adjusted cancer rates per county resulting from these models.

### Part A: Empirical Bayes
Let $N$ be the number of counties; let $y_j$ the number of kidney cancer case for the $j$-th county, $n_j$ the population of the $j$-county and $\theta_j$ be the underlying kidney cancer rate for that county. The following is a Bayesian model for our data:
\begin{aligned}
y_j &\sim Poisson(5 \cdot n_j \cdot \theta_j), \quad j = 1, \ldots, N\\
\theta_j &\sim Gamma(\alpha, \beta), \quad j = 1, \ldots, N
\end{aligned}
where $\alpha, \beta$ are hyper-parameters of the model.

(1) Using Empirical Bayes and moment matching, choose appropriate values for the hyperparameters. To do this, consider the **prior-predictive** distribution (also called evidence, the denominator normalization of bayes theorem) of the model: 

$$p(y) = \int p(y \vert \theta) p(\theta) d \theta$$.

(This is as opposed to the posterior predictive: $\int p( y \vert \theta) p(\theta \vert D) d\theta$. Not that in conjugate models such as ours, these two distributions have the same form)

For this model, the prior-predictive is a negative binomial. By matching the mean and the variance of the negative binomial to that from the data you can find $\alpha$ and $\beta$. (Hint: You need to be careful with the $5n_j$ multiplier.) Why the prior-predictive? Because you then "havent seen" individual county data yet.

(2) Produce a scatter plot of the raw cancer rates (pct mortality) vs the county population size. Highlight the top 300 raw cancer rates in red. Highlight the bottom 300 raw cancer rates in blue. Finally, on the same plot again, scatter plot the posterior mean cancer rate estimates (pct mortality) vs the county population size, highlight these in green. 

(**Note:** This is the plot (without the corrected green estimates) shown in the lecture notes for this dataset. We have provided the code to produce this plot, which you are free to use or to ignore.)


(3) Using the scatter plot, explain why using the posterior means (from our model) to estimate cancer rates is preferable to studying the raw rates themselves.

(**Hint:** You might find it helpful to review the in-class discussion on this dataset. You might also find it helpful to follow the Rat Tumor Baysian model example.)


### Part B: Full Bayes
Rather than estimating the hyperparameters $\alpha, \beta$ from the data, in a full Bayesian model we would put additional priors on quantities of which we are uncertain. That is, we can model the kidney cancer rates using a *hierarchical model*.

$$
\begin{aligned}
y_j &\sim Poisson(5 \cdot n_j \cdot \theta_j), \quad j = 1, \ldots, N\\
\theta_j &\sim Gamma(\alpha, \beta), \quad j = 1, \ldots, N\\
\alpha &\sim Gamma(a, b)\\
\beta &\sim Gamma(c, d)
\end{aligned}
$$

where $a, b, c, d$ are hyperparameters. 


(1) Analytically derive the full conditional, up constants that are inconsequential for sampling, for each variable in the model. 
  
A possible set of correct final forms of your conditionals looks like the following:
      
$$
\begin{aligned}
p(\theta_j  \vert  y_j, \mu, \sigma) &\propto Gamma\left(\theta_j; y_j + \alpha, \beta + 5n_j\right)\\
p(\alpha, \beta  \vert  y_j, \theta_j, \sigma^2) &\propto \prod_j\theta_j^{\alpha} \exp\left\{ - \beta\sum_j \theta_j \right\} * Gamma\left(\alpha; a, b \right) * Gamma\left(\beta; c, d \right) \\
\end{aligned}
$$



(2) Implement a Metropolis-Hastings-in-Gibbs sampler (like in rat tumors) to produce sets of theta-values, $\{ \theta_j: j = 1, \ldots, N\}$. Note that for updating some variables, you may need to use a Metropolis-Hastings step (see Rat Tumor example). 

  We suggest the following settings for your MH-in-Gibbs sampler, but please feel free to use your own:
  
  - Gibbs sweeps: 100,000
  - $a = 9, b = 6, c = 9, d = 0.00001$
  - Starting points: $\alpha_0 = \alpha_{\text{emperical bayes}}, \beta_0 = \beta_{\text{emperical bayes}}$
  - proposal distribution: $\mathcal{N}\left([\alpha, \beta], \left[\begin{array}{cc} 0.1 & 0 \\ 0 & 1000\end{array}\right]\right)$
  - burn in: 10%
  - thinning: 10
  
  We do not guarantee convergence after 100,000 sets of samples under these conditions. In fact, ***convergence in a untweaked sampler is highly unlikely after only 100000 Gibbs sweeps***. 

  You are responsible for performing your own convergence tests and diagnostics.

  **Note:** to sanity check your implementation, you may wish to implement this model in PyMC3 and compare your results. Note that because you model is hand tuned to deal with the conditionals and to sample straight away from the Gamma's, it will likely run faster)
  
  

- **Extra Credit:** Tweak the settings in your sampler (including number of sweeps) in order to achieve convergence. For example, do you need to sample $\alpha$ and $\beta$ jointly? Can sampling them individually speed up your sampler? Only do this after you have a complete homework. It is a rabbit hole.



(3) Using your samples, produce a pct mortality vs county population scatter plot exactly as in Part A.

In [861]:
# data_df is the dataframe of your data
# estimates is a numpy array of cancer rate estimates, one for each county
def scatter_plot_cancer_rates(data_df, estimates):
    ax = data_df.plot(kind='scatter', x="pop",y="pct_mortality", alpha=0.05, color="grey")
    # NOTE:if you have an older pandas version you should use 
    # "data_df.sort" instead of "data_df.sort_values"
    bot_kcancer_counties = data_df.sort_values(by='pct_mortality',ascending=True)[:300]
    top_kcancer_counties = data_df.sort_values(by='pct_mortality',ascending=False)[:300]
    top_kcancer_counties.plot(kind='scatter',x="pop",y="pct_mortality",alpha=0.07, color="blue", ax=ax, logx=True)
    bot_kcancer_counties.plot(kind='scatter',x="pop",y="pct_mortality",alpha=0.07, color="red", ax=ax, logx=True)
    ax.plot(data_df['pop'], 5 * estimates, '.', alpha=0.04, color="green")
    ax.set_ylim([-0.0001, 0.0003])